In [5]:
import pandas as pd
import os
import glob
from datetime import datetime

def load_yearly_data(year: int, base_path: str) -> pd.DataFrame:
    """
    Loads all daily data files for a specific year into a single pandas DataFrame.

    This function navigates to the specified year's directory, reads all '.csv.gz'
    and '.csv' files, adds a 'date' column based on the filename, and concatenates
    them into one large DataFrame.

    Args:
        year (int): The year for which to load data (e.g., 2020).
        base_path (str): The root directory containing the yearly data folders.
                         (e.g., '/path/to/your/data_directory')

    Returns:
        pd.DataFrame: A DataFrame containing all data for the specified year,
                      with an added 'date' column. Returns an empty DataFrame
                      if the directory is not found or contains no data files.
    """
    print(f"--- Starting to load data for the year: {year} ---")

    # Construct the path to the year's data directory
    year_str = str(year)
    year_path = os.path.join(base_path, year_str)

    # Check if the directory for the year exists
    if not os.path.isdir(year_path):
        print(f"Error: Directory not found at '{year_path}'")
        return pd.DataFrame()

    # Find all relevant data files (both .csv.gz and .csv)
    # Using glob is robust and finds all matching files.
    # We primarily look for .gz files as they are the standard for most days.
    # The set() will automatically handle the duplicate day (raw .csv and .gz)
    # on the first business day of the year.

    file_pattern = os.path.join(year_path, f"{year_str}*.gz")
    gz_files = glob.glob(file_pattern)

    # Also check for the uncompressed CSV for the first day of the year
    csv_pattern = os.path.join(year_path, f"{year_str}*.csv")
    csv_files = glob.glob(csv_pattern)

    # Combine lists and get unique file paths
    all_files = sorted(list(set(gz_files + csv_files)))

    if not all_files:
        print(f"No data files found in '{year_path}'")
        return pd.DataFrame()

    daily_dataframes = []
    total_files = len(all_files)

    print(f"Found {total_files} daily data files to process for {year}.")

    for i, file_path in enumerate(all_files):
        # Extract the date string (e.g., '20200102') from the filename
        # os.path.basename -> '20200102.csv.gz'
        # .split('.')[0]  -> '20200102'
        filename = os.path.basename(file_path)
        date_str = filename.split('.')[0]

        # This handles the case where we might process both the .csv and .csv.gz
        # for the first day. We only want to process each date once.
        # We check if the last processed date is the same as the current one.
        if daily_dataframes and daily_dataframes[-1]['date'].iloc[0] == pd.to_datetime(date_str, format='%Y%m%d'):
            continue

        # print(f"  ({i+1}/{total_files}) Reading file: {filename}") # This can be too verbose when loading all years

        try:
            # pandas' read_csv can automatically handle .gz compression
            # The documentation lists the column headers, which we can trust
            # are in the file.
            daily_df = pd.read_csv(file_path, compression='infer')

            # Add the date column for analysis
            daily_df['date'] = pd.to_datetime(date_str, format='%Y%m%d')

            daily_dataframes.append(daily_df)

        except Exception as e:
            print(f"    Could not read or process file {filename}. Error: {e}")

    if not daily_dataframes:
        print(f"No data was successfully loaded for {year}.")
        return pd.DataFrame()

    # Concatenate all the daily DataFrames into a single one for the whole year
    print(f"Combining daily files for {year} into a yearly DataFrame...")
    yearly_df = pd.concat(daily_dataframes, ignore_index=True)

    print(f"--- Successfully loaded {len(yearly_df):,} rows for {year}. ---")

    return yearly_df

def load_all_data(base_path: str) -> pd.DataFrame:
    """
    Loads data from all available year directories into a single DataFrame.

    This function scans the base_path for subdirectories named as years
    (e.g., '2019', '2020'), uses the `load_yearly_data` function for each,
    and concatenates all of them into one large DataFrame.

    Args:
        base_path (str): The root directory containing the yearly data folders.
                         (e.g., '/path/to/your/data_directory')

    Returns:
        pd.DataFrame: A single DataFrame containing all data from all found years.
                      Returns an empty DataFrame if the base directory is not found
                      or no valid year subdirectories are found.
    """
    print(f"=== Starting to load all data from base directory: '{base_path}' ===")
    if not os.path.isdir(base_path):
        print(f"Error: Base directory not found at '{base_path}'")
        return pd.DataFrame()

    # Find all subdirectories that look like years (e.g., '2020', '2021')
    try:
        # Use os.scandir for efficiency and filter for directories that are 4-digit numbers
        year_dirs = sorted([
            d.name for d in os.scandir(base_path)
            if d.is_dir() and d.name.isdigit() and len(d.name) == 4
        ])
    except FileNotFoundError:
        print(f"Error: Cannot access directory '{base_path}'.")
        return pd.DataFrame()

    if not year_dirs:
        print(f"No year directories (e.g., '2020', '2021') found in '{base_path}'.")
        return pd.DataFrame()

    print(f"Found year directories: {', '.join(year_dirs)}")

    all_dataframes = []
    for year_str in year_dirs:
        year = int(year_str)
        # Call the existing function to load data for one year
        yearly_df = load_yearly_data(year=year, base_path=base_path)

        if not yearly_df.empty:
            all_dataframes.append(yearly_df)
        else:
            print(f"Warning: No data was loaded for the year {year}. Skipping.")

    if not all_dataframes:
        print("No data could be loaded from any of the year directories.")
        return pd.DataFrame()

    # Concatenate all the yearly DataFrames into a single master DataFrame
    print("\n>>> Combining all yearly DataFrames into a single master DataFrame...")
    master_df = pd.concat(all_dataframes, ignore_index=True)

    print(f"=== Successfully loaded a total of {len(master_df):,} rows from {len(year_dirs)} years. ===")

    return master_df


if __name__ == '__main__':
    # =================================================================
    # HOW TO USE THE FUNCTIONS
    # =================================================================

    # 1. Set the path to the main folder containing your yearly data folders.
    #    For example, if your files are in 'D:/FinancialData/2020/20200102.csv.gz',
    #    then your base_path is 'D:/FinancialData'.
    #    (Using a raw string r'...' or forward slashes '/' is good practice for paths)
    DATA_DIRECTORY = r'C:\Users\james\Downloads\Yearly' # Example for a local 'Data/Yearly' subfolder


    # 2. Call the function to load all data from all year folders
    all_data = load_all_data(base_path=DATA_DIRECTORY)

    # 3. Inspect the resulting DataFrame
    if not all_data.empty:
        print("\n--- Final Data Inspection ---")
        print(f"Shape of the final loaded data: {all_data.shape}")

        print("\nFirst 5 rows:")
        print(all_data.head())

        print("\nLast 5 rows:")
        print(all_data.tail())

        print("\nData types and non-null counts:")
        all_data.info()

        if 'SICCD' in all_data.columns:
            print("Cleaning 'SICCD' column...")
            # Step 1: Convert to numeric, forcing errors to become 'NaN'
            all_data['SICCD'] = pd.to_numeric(all_data['SICCD'], errors='coerce')

            # Step 2: Convert the column to a nullable integer type.
            # This preserves the numbers as integers while properly handling missing values.
            all_data['SICCD'] = all_data['SICCD'].astype('Int64')
            print("'SICCD' column converted to Int64.")
        else:
            print("Column 'SICCD' not found, skipping cleaning step.")

        print("\n--- Data Inspection (After Cleaning) ---")
        # Notice the change in dtype for SICCD from 'object' to 'Int64'
        all_data.info()

        # Verify the date range
        min_date = all_data['date'].min().strftime('%Y-%m-%d')
        max_date = all_data['date'].max().strftime('%Y-%m-%d')
        print(f"\nData spans from {min_date} to {max_date}")

        # Example of a simple analysis across all years:
        if 'SPY' in all_data['ticker'].values:
            spy_avg_volume = all_data[all_data['ticker'] == 'SPY']['volume'].mean()
            print(f"\nAverage daily volume for SPY across all years: {spy_avg_volume:,.0f} shares")

=== Starting to load all data from base directory: 'C:\Users\james\Downloads\Yearly' ===
Found year directories: 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
--- Starting to load data for the year: 2000 ---
Found 253 daily data files to process for 2000.
Combining daily files for 2000 into a yearly DataFrame...
--- Successfully loaded 791,871 rows for 2000. ---
--- Starting to load data for the year: 2001 ---
Found 248 daily data files to process for 2001.
Combining daily files for 2001 into a yearly DataFrame...
--- Successfully loaded 735,575 rows for 2001. ---
--- Starting to load data for the year: 2002 ---
Found 253 daily data files to process for 2002.
Combining daily files for 2002 into a yearly DataFrame...
--- Successfully loaded 737,188 rows for 2002. ---
--- Starting to load data for the year: 2003 ---
Found 253 daily data files to process for 2003.
Combining daily files for 2003 into a yea

In [2]:
all_data.to_parquet(
    'all_data.parquet',
    compression='snappy'  # Compresses the file to save space
)

In [4]:
import pandas as pd
import os # Import the 'os' module

# Your directory path
DATA_DIRECTORY = r'C:\Users\james\ICAIF_25\Current_Code\Data'

# The name of your file
file_name = 'all_data_consolidated.parquet'

# Create the full, correct path to the file
full_path = os.path.join(DATA_DIRECTORY, file_name)

# --- Check that the path is correct before loading (optional but good practice) ---
print(f"Attempting to load file from: {full_path}")

# Now, use the full path to read the file
new = pd.read_parquet(full_path, engine='pyarrow')

print("\nFile loaded successfully!")
new.head()

Attempting to load file from: C:\Users\james\ICAIF_25\Current_Code\Data\all_data_consolidated.parquet

File loaded successfully!


,Unnamed: 0,ticker,open,high,low,close,volume,OPCL,pvCLCL,prevAdjClose,...,SICCD,PERMCO,prevRawOpen,prevRawClose,prevAdjOpen,date,volume_notional,mddv21,rhov,dhl
0,1.0,SPY,148.25000,148.25000,143.87500,145.43750,8164299.0,-0.019154,-0.009787,146.87,...,6726,46699,146.84375,146.87500,146.84,2000-01-03,NaN,NaN,NaN,NaN
1,2.0,XLF,23.71875,23.71875,22.71875,22.87500,963200.0,-0.036221,-0.037475,23.77,...,6726,34957,24.00000,23.76563,24.00,2000-01-03,NaN,NaN,NaN,NaN
2,3.0,XLB,26.67188,26.68750,25.93750,25.96875,140500.0,-0.026716,-0.022928,26.58,...,6726,34957,26.00000,26.57813,26.01,2000-01-03,NaN,NaN,NaN,NaN
3,4.0,XLK,55.62500,55.62500,53.50000,55.43750,938100.0,-0.003376,0.029002,53.88,...,6726,34957,54.15625,53.87500,54.16,2000-01-03,NaN,NaN,NaN,NaN
4,5.0,XLV,31.00000,31.00000,30.06250,30.29688,246700.0,-0.022942,-0.019221,30.89,...,6726,34957,30.59375,30.89063,30.60,2000-01-03,NaN,NaN,NaN,NaN


In [12]:
new.equals(all_data)

True